<a href="https://colab.research.google.com/github/mihuzz/MobileNetV3_train-inference/blob/main/FineTuneFaceDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd drive/MyDrive/Colab/

/content/drive/MyDrive/Colab


In [ ]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
# !git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

fatal: destination path 'vision' already exists and is not an empty directory.


In [ ]:
!pip install albumentations==0.4.6

In [ ]:
# Basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# xml library for parsing xml files
from xml.etree import ElementTree as et

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor,GeneralizedRCNN
from torchvision.models.detection import FasterRCNN, generalized_rcnn
from torchvision.models.detection.rpn import AnchorGenerator

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2


build Dataset

In [ ]:
# defining the files directory and testing directory
files_dir = "/content/drive/My Drive/Colab/Train"
test_dir = "/content/drive/My Drive/Colab/DataSet_4_test"


class MyCustomDataSet(torch.utils.data.Dataset):

    def __init__(self, files_dir, width, height, transforms=None):
        self.transforms = transforms
        self.files_dir = files_dir
        self.height = height
        self.width = width
        
        # sorting the images for consistency
        # To get images, the extension of the filename is checked to be jpg
        self.imgs = [image for image in sorted(os.listdir(files_dir))
                        if image[-4:]=='.jpg']
        
        
        # classes: 0 index is reserved for background
        self.classes = [ 0, 'mom','Miha']

    def __getitem__(self, idx):

        img_name = self.imgs[idx]
        image_path = os.path.join(self.files_dir, img_name)

        # reading the images and converting them to correct size and color    
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        # diving by 255
        # img_res = np.expand_dims(img_res, 0)
        # img_res = img_res.transpose((2,0,1))
        img_res /= 255.0
        # img_res = np.expand_dims(img_res, 0)
        # img_res = np.squeeze(img_res, 0)

        # print("img shape :", img.shape)
        # print("img res shape :", img_res.shape)
        # annotation file
        annot_filename = img_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.files_dir, annot_filename)
        
        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()
        
        # cv2 image gives size as height x width
        wt = img.shape[1]
        ht = img.shape[0]
        
        # box coordinates for xml files are extracted and corrected for image size given
        for member in root.findall('object'):
            labels.append(self.classes.index(member.find('name').text))
            
            # bounding box
            xmin = int(member.find('bndbox').find('xmin').text)
            xmax = int(member.find('bndbox').find('xmax').text)
            
            ymin = int(member.find('bndbox').find('ymin').text)
            ymax = int(member.find('bndbox').find('ymax').text)
            
            
            xmin_corr = (xmin/wt) *self.width
            xmax_corr = (xmax/wt) *self.width
            ymin_corr = (ymin/ht) *self.height
            ymax_corr = (ymax/ht) *self.height
            
            boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
        
        # convert boxes into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # getting the areas of the boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        
        labels = torch.as_tensor(labels, dtype=torch.int64)


        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        # image_id
        image_id = torch.tensor([idx])
        target["image_id"] = image_id


        if self.transforms:
            
            sample = self.transforms(image = img_res,
                                     bboxes = target['boxes'],
                                     labels = labels)
            
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
            
            
        return img_res, target

    def __len__(self):
        return len(self.imgs)


# check dataset
dataset = MyCustomDataSet(files_dir, 224,224)
print('length of dataset = ', len(dataset), '\n')

# getting the image and target for a test index.  Feel free to change the index.
img, target = dataset[138]
print(img.shape, '\n',target)

length of dataset =  650 

(224, 224, 3) 
 {'boxes': tensor([[ 46.2519,  78.1667, 163.8519, 177.9167]]), 'labels': tensor([2]), 'area': tensor([11730.6016]), 'iscrowd': tensor([0]), 'image_id': tensor([138])}


# Model

We will define a function for loading the model. We will call it later

In [ ]:
def get_object_detection_Resnet50(num_classes):

    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 

    return model
    

def get_mobile_V3(num_classes):

  model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
  return model



In [ ]:
def get_transform(train):
    
    if train:
        return A.Compose([
                            
                            A.HorizontalFlip(0.5),
                          print("img.shape train :", img.shape),
                     # ToTensorV2 converts image to pytorch tensor without div by 255
                            ToTensorV2(p=1.0),
                          print("img.shape train :", img.shape) 
                        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})
    else:
        return A.Compose([
                            ToTensorV2(p=1.0),
                            print("img.shape test :", img.shape)
                        ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})




In [ ]:
# use our dataset and defined transformations
dataset = MyCustomDataSet(files_dir, 224, 224, transforms= get_transform(train=True))
dataset_test = MyCustomDataSet(files_dir, 224, 224, transforms= get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(3)
indices = torch.randperm(len(dataset)).tolist()

# train test split
test_split = 0.2
tsize = int(len(dataset)*test_split)
dataset = torch.utils.data.Subset(dataset, indices[:-tsize])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-tsize:])


data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True, num_workers=4, 
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

#Define model parameters, choose cuda or not. 

---



---



In [ ]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


num_classes = 3


model= get_mobile_V3(num_classes)



model.to(device, dtype=torch.float32)


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
optimizer2 = torch.optim.Adam(model.parameters(), lr=1e-4)


lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)



Let's train!

In [ ]:
# training for 10 epochs
num_epochs = 6

for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

In [ ]:
%cd drive/MyDrive/Colab/

In [ ]:
model.to(device='cuda', dtype=torch.float32)
model.eval()
with torch.no_grad():
    # example_input = torch.rand(1, 3, 224, 224)
    img_path = "/content/drive/My Drive/Colab/Test_Data/image001.jpg"
    img = cv2.imread(img_path)
    print(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    img = torch.tensor(img, dtype=torch.float32)
    img /=255.0
    img = img.permute(2,0,1)
    # print(img)
    img_path2 = "/content/drive/My Drive/Colab/Test_Data/image487.jpg"
    img1 = cv2.imread(img_path2)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB).astype(np.float32)
    img1 = cv2.resize(img1, (256,256), interpolation=cv2.INTER_CUBIC)
    img1 = torch.tensor(img1, dtype=torch.float32)
    img1 /=255.0
    img1 = img1.permute(2,0,1)
    # print(img1)
    inplst = []
    inplst.append(img)
    inplst.append(img1)

    torch.save(model, 'faster_320_mobilev3.pt')

    traced_cpu = torch.jit.script(model, inplst)
    torch.jit.save(traced_cpu, "JIT_faster_mobileV3_large.pt")